In [19]:
from stravalib.client import Client

client = Client()

authorize_url = client.authorization_url(client_id=29653, redirect_uri='ec2-18-236-253-110.us-west-2.compute.amazonaws.com')
authorize_url

code = raw_input("Enter Access Token: ") 

token_response = client.exchange_code_for_token(client_id=29653, client_secret='24ea02bbb8a6ea5e1355293bb9fddb90117baf4e', code=code)
access_token = str(token_response.encode('utf-8'))

client.access_token = access_token
athlete = client.get_athlete()
# print("For {id}, I now have an access token {token}".format(id=athlete.id, token=client.access_token))
# print(client.get_athlete())
type(client.get_athlete())

Enter Access Token: 791c41062ab16b473260d47c6b43b95bfe9c1b99


stravalib.model.Athlete

In [20]:
name =  client.get_athlete().firstname + " " + client.get_athlete().lastname
city = client.get_athlete().city
print name
print city
type(client.get_athlete())

Sorabh Gupta
Jaipur


stravalib.model.Athlete

In [21]:
all_activites = list(client.get_activities())


In [29]:
print (dir(client.get_athlete_clubs()[15]))
all_IIIT_activites =  list(client.get_athlete_clubs()[15].activities)

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_activities', '_members', 'activities', 'admin', 'assert_bind_client', 'bind_client', 'city', 'club_type', 'country', 'cover_photo', 'cover_photo_small', 'description', 'deserialize', 'expand', 'featured', 'from_dict', 'id', 'log', 'member_count', 'members', 'membership', 'name', 'owner', 'private', 'profile', 'profile_medium', 'resource_state', 'sport_type', 'state', 'to_dict', 'url', 'verified']


In [32]:
print all_activites[0].map.summary_polyline

yfacDackmM_e@dLcLe@jDlQ_k@bAjLf[qM_ZbCiA`g@M`\kFdHTdQbDrDxPtNtDpnBiv@`\c]jvAwg@mDePfDco@hDiSi@yMrBuNiB{[Eei@c@}A_KoCiCkFK{DtB{FzCeCpGY`F`DrAfKlPlClMtFtOeNd@cYqGFoUrFpWcHfFhA[hRAwScHXuc@xKyBjBb@xImBxGiJjDqH_DcDmLiAg@uyCof@gA|@UnJxCvNnJjNnOnJ~A`E{Bd_@cLpd@mYdU_LvPmHrQiLrL


In [4]:
len(all_activites)
# for activity in all_activites:
#     print activity.map.summary_polyline
# (all_activites[9].map.summary_polyline)

298

In [18]:
all_polylines = [activity.map.summary_polyline for activity in all_activites if activity.map.summary_polyline]
type(all_polylines[0])

str

In [6]:
len(all_polylines)

284

In [7]:
LATLONG = []

import polyline
for item in all_polylines:
    coords = polyline.decode(str(item))
    for x in coords:
        LATLONG.append(x)

In [8]:
import os
import folium
import operator

print(folium.__version__)

def dict_sort_des(diction):
    sorted_d = sorted(diction.items(), key=operator.itemgetter(1),reverse=True)
    return sorted_d

0.6.0


In [9]:
data = []
for i in LATLONG:
    data.append( list(i))

In [10]:
data
frequency_dict = {}
for k in LATLONG:
    l = round(k[0], 3),round(k[1], 3)
    if l in frequency_dict.keys():
        frequency_dict[l] += 1
    else:
        frequency_dict[l] = 1
address = dict_sort_des(frequency_dict)[:5][0][0]
#List fo frequent locations

In [16]:
from folium.plugins import HeatMap

m = folium.Map(address, tiles='openstreetmap', zoom_start=13)
m.save("hyd.html")
HeatMap(data).add_to(m)

folium.plugins.HeatMap(data, radius = 20, min_opacity = 0.1, max_val = 40,gradient={.6: 'blue', .98: 'lime', 1: 'red'}).add_to(m)

m.save(os.path.join(str(name)+'heatmap.html'))

m


In [12]:
map = folium.Map(location=address, zoom_start=20)
folium.Marker(address).add_to(map)
map.save(os.path.join(str(name)+'address.html'))
map


In [13]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Strava")
location = geolocator.reverse(address)
print(location.address)

New Sanganer Road, Devi Nagar, Bapu Nagar, Jaipur Municipal Corporation, Jaipur, Rajasthan, 302015, India


In [14]:
startCoords = []
endCoords = []

temp1 = [list(polyline.decode(str(i))[0]) for i in all_polylines]
temp2 = [list(polyline.decode(str(i))[-1]) for i in all_polylines]


for i in temp1:
    startCoords.append([round(i[0],3),round(i[1],3)])

    
    
for i in temp2:
    endCoords.append([round(i[0],3),round(i[1],3)])


In [15]:
from folium.plugins import MarkerCluster


m = folium.Map(location=address, zoom_start=12)

marker_cluster = MarkerCluster().add_to(m)

for start in startCoords:
    folium.Marker(
        location=start,
        popup='Start',
        icon=folium.Icon(color='green', icon='ok-sign'),
    ).add_to(marker_cluster)

for end in endCoords:
    folium.Marker(
        location=end,
        popup='End',
        icon=folium.Icon(color='red', icon='remove-sign'),
    ).add_to(marker_cluster)

folium.Marker(
        location=address,
        popup='Predicted address',
        icon=folium.Icon(color='blue'),
    ).add_to(marker_cluster)


m.save(os.path.join(str(name)+'proof.html'))
m